In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [2]:
with open("war_text.pickle", "rb") as file:
    war_text = pickle.load(file)

In [3]:
all_text = ''.join(war_text)

In [4]:
bul_symbols = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЪЮЯабвгдежзийклмнопрстуфхцчшщьъюя"

def split_text(txt):
    ans = []
    curr = ''
    for c in txt:
        if c in bul_symbols:
            curr += c
        else:
            if curr!='': ans.append(curr)
            ans.append(c)
            curr = ''
    if curr!='': ans.append(curr)
    return ans

splited_text = split_text(all_text)
splited_text[:10]

['Кримската', ' ', 'война', ' ', '(', 'октомври', ' ', '1', '8', '5']

In [5]:
id2word = sorted(set(splited_text))
word2id = {w:i for i,w in enumerate(id2word)}
vocab_size = len(id2word)
vocab_size

38912

In [6]:
window_size = 8
embedding_dim = 128

cbow_model = tf.keras.Sequential()
cbow_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape = (window_size-1,)))
cbow_model.add(tf.keras.layers.GlobalAveragePooling1D())
cbow_model.add(tf.keras.layers.Dropout(0.4))
cbow_model.add(tf.keras.layers.Dense(vocab_size))

cbow_model.load_weights('./cbow\\ckpt_10')

cbow_model.get_weights()[0].shape

(38912, 128)

In [7]:
rnn_units = 1024
rnn_units2 = 128

def build_model(batch_size=64):
    embed = tf.keras.layers.Embedding(vocab_size, 
                                      embedding_dim, 
                                      batch_input_shape = [batch_size, None], 
                                      embeddings_initializer = tf.constant_initializer(cbow_model.get_weights()[0]), 
                                      trainable=False)
    model = tf.keras.Sequential()
    model.add(embed)
    model.add(tf.keras.layers.GRU(rnn_units, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.GRU(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.GRU(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

model = build_model(1)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            4980736   
_________________________________________________________________
gru (GRU)                    (1, None, 1024)           3545088   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 128)            443136    
_________________________________________________________________
gru_2 (GRU)                  (1, None, 128)            99072     
_________________________________________________________________
dense_1 (Dense)              (1, None, 38912)          5019648   
Total params: 14,087,680
Trainable params: 9,106,944
Non-trainable params: 4,980,736
_________________________________________________________________


In [8]:
model.load_weights(tf.train.latest_checkpoint('./word_rnn_embed'))
model.build(tf.TensorShape([1, None]))

In [9]:
def generate_text(model, start_string, num_generate = 200, temperature = 1.0):

    input_eval = [word2id[s] for s in split_text(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(id2word[predicted_id])

    return (start_string + ''.join(text_generated))

In [13]:
print(generate_text(model, "Адолф Хитлер е ", 200, 0.3))

Адолф Хитлер е убит на Византийската република е въоръжен втората в порядъка на България в Тристранния пакт войни. На 1 май 19 май 19 ноември 19 май 19 ноември 19 декември 19 ноември 19 ноември 19 септември 19 май 1919 г. до 19 май 1915 г. в битката при Сталинград (19 януари 1711 г. до 19 септември 19 октомври – май май 1946 г. до 16 април – 19 декември 19 януари 1919 г. в двореца на Третия райх България (19 януари 18
